<a href="https://colab.research.google.com/github/daveshap/QuestionDetector/blob/main/DownloadGutenbergTop100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download Top 100 Ebooks
Gutenberg has a bunch of books for free. Fiction has a lot of dialog. 

In [ ]:
import requests

prefix = 'https://www.gutenberg.org/files/'
# final address example: https://www.gutenberg.org/files/84/84-0.txt
books = ['84','1342','25344','2542','11','1952','16328','1250','41','1080','46','5200','98','43','1232','1661','2701','345','844','23','160','209','205','76','1064','42108','408','74','1260','174','2591','16','1497','1400','2852','120','219','215','42884','3207','6130','42324','38269','158','203','2814','514','2500','3825','2600','40074','55','4300','58585','15399','45','1184','36','5740','1727','140','768','902','2554','996','113','3600','2148','19942','57775','244','27827','135','28054','7370','132','20203','13701','1254','1001','1063','11030','32415','779','3296','4363','34901','22381','12122','236','308','4517','35','376','147','1251','766','16643','1998','730']

for book in books:
  bookurl = prefix + '%s/%s-0.txt' % (book, book)
  response = requests.get(bookurl)
  body = response.text
  with open('/content/drive/My Drive/Gutenberg/%s.txt' % book, 'w', encoding='utf-8') as file:
    file.write(body)

# Cleanup Docs
- Split books based on sections delimited by vertical whitespace
- Remove short sections, sections full of all caps, and sections with too many symbols

In [ ]:
import os
import re

gut_dir = '/content/drive/My Drive/Gutenberg/'
data = list()

def clean_chunk(text):
  text = text.strip()  # trim the ends
  text = re.sub(r'\t', ' ', text)  # replace tabs with spaces
  text = re.sub(r'\s+', ' ', text)  # replace any extra whitespace with single
  return text
  
def measure_text(text)  :
  # returns count of: chars, word chars, digits, whitespace
  pct_wordchars = len(re.findall('\w', text)) / len(text)
  pct_digits = len(re.findall('\d', text)) / len(text)
  pct_whitespace = len(re.findall('\s', text)) / len(text)
  pct_caps = len(re.findall('[A-Z]', text)) / len(text)
  return pct_wordchars, pct_digits, pct_whitespace, pct_caps

for file in os.listdir(gut_dir):
  with open(gut_dir + file, 'r', encoding='utf-8') as file:
    body = file.read()
  # TODO check if \n\n exists or not
  chunks = re.split(r'[\r\n]{4,}', body)
  for chunk in chunks:
    # TODO check chunk for all caps, too many symbols, quotation marks, etc
    if '"' in chunk or '“' in chunk:  # if text is dialog, put it in
      data.append(clean_chunk(chunk))
      continue
    if '!' in chunk or '?' in chunk:
      data.append(clean_chunk(chunk))
      continue
    pwc, pd, pws, pc = measure_text(chunk)
    if pwc < 0.8:  # if too many symbols
      continue
    if pc > 0.6:  # if too many caps
      continue 
    data.append(clean_chunk(chunk))